In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [3]:

data = pd.read_pickle('../create_SL_data/data_18months_levels_ua.pkl')

data.dropna(inplace=True)

data

,patient_number,cluster_0_size-diff%_pre_to_3h,cluster_1_size-diff%_pre_to_3h,cluster_2_size-diff%_pre_to_3h,cluster_3_size-diff%_pre_to_3h,cluster_4_size-diff%_pre_to_3h,cluster_5_size-diff%_pre_to_3h,cluster_6_size-diff%_pre_to_3h,cluster_7_size-diff%_pre_to_3h,cluster_8_size-diff%_pre_to_3h,...,q90_difference_pre_7d_cluster_19_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_20_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_21_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_22_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_23_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_24_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_25_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_26_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_27_158Gd_pSTAT3_Y705,BCRABL
0,01,112.311228,4.657895,193.583600,17.357513,131.736527,41.726700,-9.016142,581.484794,341.481481,...,-0.036379,0.538215,0.244545,0.173798,0.520689,0.000000,0.445879,0.528887,-0.634862,0.0000
1,02,-39.227196,-35.480393,-44.449358,-42.916013,-23.653129,-27.315147,-71.392762,-4.200082,-66.785333,...,0.409531,0.471072,0.520922,0.195533,0.814652,0.000000,0.649210,0.523819,-0.824650,0.0000
2,03,107.606356,226.932177,32.365967,344.897484,21.987033,155.866808,96.242718,139.396945,20.366891,...,0.033095,-0.010136,0.182196,0.013442,0.234621,0.450780,0.509549,0.256670,-2.014077,0.0310
3,04,-25.682913,124.070561,135.671642,44.396807,-1.134931,83.993576,2.053080,36.713996,-25.974026,...,0.197905,-0.030211,0.115321,0.107060,0.578006,0.125471,0.312356,0.236907,-1.469164,0.2780
4,05,201.201479,43.747082,122.874251,43.729788,-36.776933,182.597569,-3.854390,119.651741,6.158358,...,0.528722,0.116360,0.030627,0.588062,0.544317,0.414612,0.000000,0.000000,-0.530531,0.2700
5,06,1.924515,15.800274,30.471903,-24.722838,-0.323767,83.695906,-34.709775,-28.399925,50.686604,...,0.228519,-0.704528,0.353815,0.126909,0.615345,0.805617,0.184870,0.000000,-1.011029,0.0520
7,08,-31.193715,-35.195035,25.087948,-39.439962,-17.761805,-27.302254,-34.826649,-12.109273,-26.896508,...,0.085245,-0.113622,0.375774,0.155500,-0.032117,0.751987,0.243146,0.494612,-1.508415,0.0260
8,09,12.781223,-20.827075,54.025045,-10.660076,100.369004,-8.090675,-24.769019,-14.171028,209.620596,...,0.838215,0.285612,0.294480,0.515315,0.613103,1.708393,-0.129841,0.539467,-1.008428,0.0120
9,11,-20.563531,27.474541,24.549279,-6.790533,175.652174,2.324571,6.422837,-6.846860,289.677419,...,0.683557,-0.295606,0.232018,0.329252,0.860805,0.000000,-0.124076,-0.068443,-0.809707,0.0000
10,12,60.052338,33.746492,35.745788,103.864279,334.647303,35.013295,2.785379,60.158979,150.288184,...,0.421078,0.452105,0.311938,0.148396,0.537945,1.074841,-0.339275,0.085685,-2.497944,0.0470


In [4]:
data['patient_number'].unique()

array(['01', '02', '03', '04', '05', '06', '08', '09', '11', '12', '13',
       '14', '15', '16', '17', '18', '22', '23', '24', '25', '26', '27',
       '28', '29', '32', '33', '34', '35', '36', '37', '38'], dtype=object)

In [5]:
X = data.drop(['BCRABL', 'patient_number'], axis=1)  
y = data['BCRABL']                                 

patient_numbers = data['patient_number']

## Leave one out nested cross validation

In [24]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import mean_squared_error

models = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 10, 50]}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
    },
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}  
    }
}

feature_importances = {name: pd.DataFrame(np.zeros((X.shape[0], X.shape[1])), columns=X.columns) for name in models if name == 'RandomForest'}
scores = {name: {'mse_scores': [], 'rmse_scores': [], 'best_params': []} for name in models}

outer_cv = LeaveOneOut()
inner_cv = LeaveOneOut()

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, config in models.items():
        if config['params']:
            grid_search = GridSearchCV(config['model'], config['params'], cv=inner_cv, scoring='neg_mean_squared_error')
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            scores[name]['best_params'].append(grid_search.best_params_)
        else:
            best_model = config['model']
            best_model.fit(X_train, y_train)
            scores[name]['best_params'].append({})

        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        scores[name]['mse_scores'].append(mse)
        scores[name]['rmse_scores'].append(rmse)

        if name == 'RandomForest':
            feature_importances[name].iloc[fold_idx, :] = best_model.feature_importances_

    print(f'Fold {fold_idx + 1} completed')

results = {}
for name in models:
    average_mse = np.mean(scores[name]['mse_scores'])
    average_rmse = np.mean(scores[name]['rmse_scores'])
    results[name] = {
        'Average MSE': average_mse,
        'Average RMSE': average_rmse,
        'Best Parameters': scores[name]['best_params'][-1]
    }

    if name == 'RandomForest':
        mean_fi = feature_importances[name].mean(axis=0).sort_values(ascending=False)
        results[name]['Feature Importances'] = mean_fi

for name, result in results.items():
    print(f"{name} Results:")
    for key, value in result.items():
        print(f"{key}: {value}")


Fold 1 completed
Fold 2 completed
Fold 3 completed
Fold 4 completed
Fold 5 completed
Fold 6 completed
Fold 7 completed
Fold 8 completed
Fold 9 completed
Fold 10 completed
Fold 11 completed
Fold 12 completed
Fold 13 completed
Fold 14 completed
Fold 15 completed
Fold 16 completed
Fold 17 completed
Fold 18 completed
Fold 19 completed
Fold 20 completed
Fold 21 completed
Fold 22 completed
Fold 23 completed
Fold 24 completed
Fold 25 completed
Fold 26 completed
Fold 27 completed
Fold 28 completed
Fold 29 completed
Fold 30 completed
Fold 31 completed
RandomForest Results:
Average MSE: 0.008399358430360198
Average RMSE: 0.06871553063595064
Best Parameters: {'max_depth': 10, 'n_estimators': 200}
Feature Importances: q90_difference_pre_7d_cluster_7_150Nd_pSTAT5_Y694    7.516371e-02
90th_143Nd_pCRKL Y207_3h_cluster_24                  3.702054e-02
q90_difference_pre_3h_cluster_7_150Nd_pSTAT5_Y694    3.609283e-02
90th_158Gd_pSTAT3 Y705_3h_cluster_16                 3.279794e-02
90th_172Yb_pS6 S235S

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

In [7]:
model1 = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
model2 = KNeighborsRegressor(n_neighbors=7, weights='uniform')

model1.fit(X, y)
model2.fit(X, y)

KNeighborsRegressor(n_neighbors=7)

In [8]:
model1.score(X, y)

0.853253881619262

In [9]:
model2.score(X, y)

0.17054187836008505

In [10]:
y.describe()

count    31.000000
mean      0.055274
std       0.086107
min       0.000000
25%       0.002800
50%       0.015000
75%       0.049500
max       0.278000
Name: BCRABL, dtype: float64

In [11]:
pd.set_option('display.max_colwidth', None)

coefficients = model1.feature_importances_  

feature_importance = pd.DataFrame(data={'Feature': X.columns, 'Importance': coefficients})

feature_importance['Absolute Importance'] = feature_importance['Importance'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Importance', ascending=False)

feature_importance.head(30)

,Feature,Importance,Absolute Importance
2051,q90_difference_pre_7d_cluster_7_150Nd_pSTAT5_Y694,0.092113,0.092113
1631,q90_difference_pre_3h_cluster_7_150Nd_pSTAT5_Y694,0.045969,0.045969
688,90th_158Gd_pSTAT3 Y705_3h_cluster_16,0.031953,0.031953
415,90th_172Yb_pS6 S235S236_pre_cluster_23,0.029189,0.029189
528,90th_143Nd_pCRKL Y207_3h_cluster_24,0.023578,0.023578
2046,q90_difference_pre_7d_cluster_2_150Nd_pSTAT5_Y694,0.022712,0.022712
1397,q90_difference_pre_3h_cluster_25_176Yb_pS6_S240244,0.021731,0.021731
920,90th_142Nd_cCaspase3_7d_cluster_24,0.016977,0.016977
1728,q90_difference_pre_3h_cluster_20_158Gd_pSTAT3_Y705,0.015068,0.015068
1167,90th_165Ho_pCREB S133_7d_cluster_19,0.014862,0.014862


In [42]:
data_pool2 = pd.read_pickle('../create_SL_data_pool2/data_18months_levels_ua_pool2.pkl')
pool2_patients = pd.DataFrame(data_pool2['patient_number'])



In [43]:
X_test = data_pool2.drop(['BCRABL', 'patient_number'], axis=1)  
y_test = data_pool2['BCRABL']                                 

y_test

0    0.007
1    0.830
2      NaN
3    0.150
4    0.003
5    0.002
6    1.100
7    0.020
8    0.005
Name: BCRABL, dtype: float64

In [44]:
model1.predict(X_test)

array([0.0687695, 0.1032465, 0.0783245, 0.0684315, 0.0714515, 0.079823 ,
       0.0490475, 0.10667  , 0.0625815])

In [45]:
pool2_patients['pred'] = model1.predict(X_test)
pool2_patients['month'] = 18
pool2_patients['patient_number'] = pool2_patients['patient_number'].astype(int)

pool2_patients

,patient_number,pred,month
0,39,0.068770,18
1,40,0.103246,18
2,41,0.078324,18
3,42,0.068431,18
4,44,0.071451,18
5,45,0.079823,18
6,54,0.049047,18
7,56,0.106670,18
8,57,0.062581,18


In [46]:
patient_numbers_pool2 = data_pool2['patient_number'].astype(int)

response = pd.read_csv('../response/responses_all.csv')
response_pool1 = response[response['patient_number'].isin(patient_numbers_pool2)]

response_pool1

,patient_number,patient_id,batch,month,BCRABL
237,39,4810_00009,5,3,0.070
238,39,4810_00009,5,6,0.020
239,39,4810_00009,5,9,0.020
240,39,4810_00009,5,12,0.007
241,39,4810_00009,5,15,0.009
...,...,...,...,...,...
295,57,4306_00004,6,9,0.025
296,57,4306_00004,6,12,NaN
297,57,4306_00004,6,15,NaN
298,57,4306_00004,6,17,0.007


In [47]:
merged_data = pd.merge(response_pool1, pool2_patients, on=['patient_number', 'month'], how='outer')
merged_data


,patient_number,patient_id,batch,month,BCRABL,pred
0,39,4810_00009,5,3,0.070,NaN
1,39,4810_00009,5,6,0.020,NaN
2,39,4810_00009,5,9,0.020,NaN
3,39,4810_00009,5,12,0.007,NaN
4,39,4810_00009,5,15,0.009,NaN
...,...,...,...,...,...,...
58,57,4306_00004,6,9,0.025,NaN
59,57,4306_00004,6,12,NaN,NaN
60,57,4306_00004,6,15,NaN,NaN
61,57,4306_00004,6,17,0.007,NaN


In [48]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data['patient_number'].unique()):
    patient_data = merged_data[merged_data['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        # Dashed lines to predicted values (before and after)
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.01)

fig.show()



In [49]:
pool1_patients = pd.DataFrame(data['patient_number'])
pool1_patients['patient_number'] = pool1_patients['patient_number'].astype(int)
pool1_patients['pred'] = model1.predict(X)
pool1_patients['month'] = 18

pool1_patients

,patient_number,pred,month
0,1,0.023252,18
1,2,0.023701,18
2,3,0.035164,18
3,4,0.202931,18
4,5,0.197369,18
5,6,0.061819,18
7,8,0.038262,18
8,9,0.022279,18
9,11,0.018124,18
10,12,0.057115,18


In [50]:
merged_data = pd.merge(response, pool1_patients, on=['patient_number', 'month'], how='outer')
merged_data

merged_data['BCRABL'] = merged_data['BCRABL'].replace(0, 0.001)

In [56]:
merged_data_1 = merged_data[merged_data['batch'] == 1]
merged_data_2 = merged_data[merged_data['batch'] == 2]
merged_data_3 = merged_data[merged_data['batch'] == 3]
merged_data_4 = merged_data[merged_data['batch'] == 4]

In [57]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_1['patient_number'].unique()):
    patient_data = merged_data_1[merged_data_1['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values Batch 1',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log' 
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.1)

fig.show()


In [58]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  
fig = go.Figure()

for idx, patient in enumerate(merged_data_2['patient_number'].unique()):
    patient_data = merged_data_2[merged_data_2['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values Batch 2',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.1)

fig.show()


In [59]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly 
fig = go.Figure()


for idx, patient in enumerate(merged_data_3['patient_number'].unique()):
    patient_data = merged_data_3[merged_data_3['patient_number'] == patient]
    patient_color = colors[idx % len(colors)] 

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values Batch 3',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.1)

fig.show()


In [60]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_4['patient_number'].unique()):
    patient_data = merged_data_4[merged_data_4['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color), 
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False 
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values Batch 4',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log' 
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.1)

fig.show()
